In [ ]:
from typing import Dict
import json
import datetime


now = datetime.datetime.now()
now = now.strftime('%Y-%m-%d %H:%M:%S')

input_path = './data/medical/ufo/train.json'
output_path = './data/medical/ufo/train_coco.json'